In [1]:
import pandas as pd
import numpy as np
import uuid

# Fungsi bantu untuk membuat ID unik jika tidak tersedia
def generate_id(prefix):
    return prefix + "_" + str(uuid.uuid4())[:8]

# 1. Load Data Mentah
# Sesuaikan path jika file ada di folder tertentu
df_authors_raw = pd.read_csv("author.csv")
df_works_openalex = pd.read_csv("openalex_works.csv")
df_works_crossref = pd.read_csv("crossref_works.csv")
df_jurnal_penelitian = pd.read_csv("jurnal_penelitian.csv")
df_issn_clean = pd.read_csv("cleaned_crossref_issn_multiple.csv")
df_stats = pd.read_csv("index_penelitian.csv")
df_dosen_works = pd.read_csv("Penelitian Berdasarkan Nama Dosen.csv")

print("Data berhasil dimuat.")
print(f"Total Authors Raw: {len(df_authors_raw)}")
print(f"Total Works (OpenAlex): {len(df_works_openalex)}")
print(f"Total Works (Jurnal Penelitian): {len(df_jurnal_penelitian)}")

Data berhasil dimuat.
Total Authors Raw: 65
Total Works (OpenAlex): 3161
Total Works (Jurnal Penelitian): 1422


In [2]:
# Ambil kolom penting dari author.csv
dim_authors = df_authors_raw[['author_id', 'name', 'institution', 'orcid']].copy()

# Pastikan author_id unik (hapus duplikat jika ada)
dim_authors = dim_authors.drop_duplicates(subset=['author_id'])

# Kita coba ambil mapping NIDN dari file 'Penelitian Berdasarkan Nama Dosen.csv'
# File ini punya kolom 'nidn' dan 'fullname'
nidn_map = df_dosen_works[['nidn', 'fullname']].drop_duplicates().dropna()

# Normalisasi nama untuk pencocokan (lowercase, strip)
dim_authors['name_clean'] = dim_authors['name'].str.lower().str.strip()
nidn_map['name_clean'] = nidn_map['fullname'].str.lower().str.strip()

# Merge untuk mendapatkan NIDN ke tabel author utama
dim_authors = pd.merge(dim_authors, nidn_map[['nidn', 'name_clean']], on='name_clean', how='left')

# Rapikan kolom
dim_authors = dim_authors[['author_id', 'name', 'nidn', 'institution', 'orcid']]
dim_authors = dim_authors.drop_duplicates(subset=['author_id'])

# Simpan
dim_authors.to_csv("dim_authors.csv", index=False)
print(f"dim_authors created with {len(dim_authors)} rows.")
dim_authors.head()

dim_authors created with 65 rows.


,author_id,name,nidn,institution,orcid
0,https://openalex.org/A5048632251,Eddy Suryanto Soegoto,410036002.0,NaN,NaN
1,https://openalex.org/A5052898875,Irawan Afrianto,405047901.0,NaN,https://orcid.org/0000-0003-3245-6793
2,https://openalex.org/A5077761860,Senny Luckyardi,420028204.0,NaN,https://orcid.org/0000-0001-9954-7433
3,https://openalex.org/A5030771199,Cherry Dharmawan,420017201.0,NaN,https://orcid.org/0000-0002-5221-8787
4,https://openalex.org/A5056478165,Taryana Suryana,416126901.0,NaN,NaN


In [3]:
# Kumpulkan data jurnal dari berbagai sumber
# Prioritas 1: cleaned_crossref_issn_multiple (karena datanya lengkap dengan ISSN)
venues_1 = df_issn_clean[['Container_Title', 'Publisher', 'ISSN', 'ISSN_Electronic']].copy()
venues_1 = venues_1.rename(columns={'Container_Title': 'venue_name', 'Publisher': 'publisher', 'ISSN': 'issn', 'ISSN_Electronic': 'e_issn'})

# Prioritas 2: jurnal_penelitian (ambil nama jurnal dan iss jika ada)
venues_2 = df_jurnal_penelitian[['journal', 'publisher', 'issn']].copy()
venues_2 = venues_2.rename(columns={'journal': 'venue_name'})

# Gabungkan
all_venues = pd.concat([venues_1, venues_2])

# Bersihkan
all_venues['venue_name'] = all_venues['venue_name'].str.strip()
all_venues = all_venues.dropna(subset=['venue_name']) # Hapus yang tidak ada nama jurnalnya
all_venues = all_venues.drop_duplicates(subset=['venue_name'])

# Buat Venue ID Unik
all_venues['venue_id'] = [generate_id("VEN") for _ in range(len(all_venues))]

# Simpan
all_venues.to_csv("dim_venues.csv", index=False)
print(f"dim_venues created with {len(all_venues)} rows.")
all_venues.head()

dim_venues created with 31 rows.


,venue_name,publisher,issn,e_issn,venue_id
0,Komputika : Jurnal Sistem Komputer,Universitas Komputer Indonesia,2655-3198; 2252-9039,2655-3198,VEN_974cd0a1
195,Jurnal Agregasi : Aksi Reformasi Government da...,Universitas Komputer Indonesia,2579-3047; 2337-5299,2579-3047,VEN_e9d6d125
305,Jurnal Manajemen Informatika (JAMIKA),Universitas Komputer Indonesia,2655-6960; 2088-4125,2655-6960,VEN_e99ec3eb
496,Jurnal Teknologi dan Informasi,Universitas Komputer Indonesia,2655-6839; 2088-2270,2655-6839,VEN_444398b3
621,Jurnal Common,Universitas Komputer Indonesia,2580-6386; 2654-9271,2580-6386,VEN_98ee4175


In [6]:
# Kita ambil data works dari jurnal_penelitian (paling lengkap strukturnya)
works_main = df_jurnal_penelitian[['work_id', 'title', 'doi', 'year', 'abstract', 'journal']].copy()

# Kita ambil juga dari OpenAlex Works jika work_id belum ada di main
# (Dalam kasus ini kita asumsikan merge sederhana, di real case butuh complex deduplication)
works_oa = df_works_openalex[['title', 'year', 'journal', 'type', 'open_access']].copy()
works_oa['work_id'] = [generate_id("WORK") for _ in range(len(works_oa))] # Generate ID sementara

# Standarisasi Kolom
works_main['source'] = 'jurnal_penelitian'
works_oa['source'] = 'openalex'

# Gabungkan (Concatenate)
# Catatan: Idealnya kita cek duplikasi berdasarkan DOI atau Judul yang dinormalisasi
all_works = pd.concat([works_main, works_oa], ignore_index=True)

# Deduplikasi berdasarkan Title (karena DOI tidak selalu ada)
all_works['title_clean'] = all_works['title'].str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True)
all_works = all_works.drop_duplicates(subset=['title_clean'])

# LINK KE VENUE: Ganti nama jurnal dengan venue_id
# Load venue map
venue_map = all_venues[['venue_name', 'venue_id']].copy()
venue_map['venue_name_clean'] = venue_map['venue_name'].str.lower().str.strip()

# Convert journal column to string and handle NaN values
all_works['journal_clean'] = all_works['journal'].fillna('').astype(str).str.lower().str.strip()

# Merge with venue map (left join to keep all works even without matching venue)
all_works = pd.merge(all_works, venue_map[['venue_name_clean', 'venue_id']], 
                     left_on='journal_clean', right_on='venue_name_clean', how='left')

# Rapikan kolom akhir
final_works = all_works[['work_id', 'title', 'doi', 'year', 'venue_id', 'type', 'abstract']].copy()

# Isi work_id yang kosong (dari source yang tidak punya ID)
final_works['work_id'] = final_works['work_id'].apply(lambda x: x if pd.notna(x) else generate_id("WORK"))

final_works.to_csv("dim_works.csv", index=False)
print(f"dim_works created with {len(final_works)} rows.")
final_works.head()

dim_works created with 2836 rows.


,work_id,title,doi,year,venue_id,type,abstract
0,https://openalex.org/W4210647064,Designing an Educational Website regarding Rec...,https://doi.org/10.17509/ajsee.v1i3.33755,2021,NaN,NaN,Plastic waste is closely related to the enviro...
1,https://openalex.org/W3215723935,Kompetensi wirausaha dan gaya kepemimpinan ter...,NaN,2014,NaN,NaN,The research was conducted on the owner or man...
2,https://openalex.org/W2784401774,Metode riset online,NaN,2014,NaN,NaN,Buku ini menjawab persoalan riset online yang ...
3,https://openalex.org/W2210614275,PERBANDINGAN METODE JARINGAN SYARAF TIRUAN BAC...,NaN,2012,NaN,NaN,Pengenalan wajah merupakan suatu bidang yang m...
4,https://openalex.org/W2756237186,RANCANG BANGUN APLIKASI CHATBOT INFORMASI OBJE...,https://doi.org/10.34010/komputa.v4i1.2410,2015,NaN,NaN,Informasi adalah bagian penting dalam kehidupa...


In [7]:
# Menggunakan index_penelitian.csv
fact_stats = df_stats.copy()

# Pastikan kolom sesuai tipe data
cols_to_num = ['total_publications', 'total_citations', 'h_index', 'i10_index']
for col in cols_to_num:
    fact_stats[col] = pd.to_numeric(fact_stats[col], errors='coerce').fillna(0)

# Simpan
fact_stats.to_csv("fact_author_stats.csv", index=False)
print("fact_author_stats created.")
fact_stats.head()

fact_author_stats created.


,author_id,total_publications,total_citations,h_index,i10_index,avg_citations
0,https://openalex.org/A5048632251,3,9,1,0,3.00
1,https://openalex.org/A5052898875,76,461,12,17,6.07
2,https://openalex.org/A5077761860,67,306,11,12,4.57
3,https://openalex.org/A5030771199,14,20,3,0,1.43
4,https://openalex.org/A5056478165,45,44,4,2,0.98


In [8]:
# 1. Hubungan dari jurnal_penelitian.csv (sudah ada work_id dan author_id)
rels_1 = df_jurnal_penelitian[['work_id', 'author_id']].dropna()

# 2. Hubungan dari Penelitian Dosen (NIDN based)
# Kita perlu map NIDN ke Author_ID dulu
nidn_to_author_id = dim_authors[['nidn', 'author_id']].dropna().set_index('nidn')['author_id'].to_dict()

# Ambil data dosen works
rels_2_raw = df_dosen_works[['nidn', 'title']].copy()
# Cari work_id berdasarkan judul (merge dengan dim_works yang baru dibuat)
rels_2_raw['title_clean'] = rels_2_raw['title'].str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True)
works_lookup = final_works[['title', 'work_id']].copy()
works_lookup['title_clean'] = works_lookup['title'].str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True)

rels_2 = pd.merge(rels_2_raw, works_lookup[['title_clean', 'work_id']], on='title_clean', how='inner')

# Map NIDN ke Author ID
rels_2['author_id'] = rels_2['nidn'].map(nidn_to_author_id)
rels_2 = rels_2[['work_id', 'author_id']].dropna()

# Gabungkan semua relationship
all_rels = pd.concat([rels_1, rels_2])
all_rels = all_rels.drop_duplicates()

# Simpan
all_rels.to_csv("fact_authorship.csv", index=False)
print(f"fact_authorship created with {len(all_rels)} connections.")
all_rels.head()

fact_authorship created with 2625 connections.


,work_id,author_id
0,https://openalex.org/W4210647064,https://openalex.org/A5048632251
1,https://openalex.org/W3215723935,https://openalex.org/A5048632251
2,https://openalex.org/W2784401774,https://openalex.org/A5048632251
3,https://openalex.org/W2210614275,https://openalex.org/A5052898875
4,https://openalex.org/W2756237186,https://openalex.org/A5052898875
